In [15]:
import tensorflow as tf
from tensorflow.keras.layers import  Dropout , Flatten , Dense ,GlobalAveragePooling2D,Lambda,Input
from tensorflow.keras import models , optimizers , regularizers
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from PIL import Image
from tensorflow.keras.models import Sequential
import numpy as np


In [2]:
#Método 1: nbimporter permite importar notebooks directamente.
# notebook_principal.ipynb
import nbimporter

# Importa la clase desde notebook_clase
from Rescale import reScale
#Método 2: %run ejecuta otro notebook y hace accesibles las variables y clases definidas en él.
#%run 'Rescale.ipynb'

In [ ]:
class model():

    def __init__(self,image):
        self.image = image

    def creationModel():
        # Creation of a instance of pre-train model ResNet50
        ResNet50_conv_base  = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        #Do not retrain layers that are already trained
        for layer in ResNet50_conv_base.layers:
            layer.trainable = False

        # CNN Model
        model = Sequential([

        # Input Layer
        Input(shape=(224,224,3)),

        # ResNet Preprocessing (applied during training & inference)
        Lambda(preprocess_input),

        ResNet50_conv_base,    
        
        GlobalAveragePooling2D(),
            
        Flatten(),

        Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),

        # The final `Dense` layer with the number of classes.
        Dense(1, activation ='sigmoid')
        ])

        return model
    
    def loadModel(self):
        # Optimizer
        opt = optimizers.AdamW(learning_rate=0.0001, weight_decay=1e-5)

        #Copy architecture of the model
        model_best = self.creationModel()

        #Charge the best weigths to model_best
        model_best.load_weights('weights/Best_Model.keras')

        #Compile the model
        model_best.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

        return model_best
    
    def prediction(self):
        model_best = self.loadModel() #charge the best weight and architercture
        obj_imagen = reScale(self.image) # instance of the class reScale
        imagen = obj_imagen.changeScale() # call the function and return the rescale image
        y_predict = model_best.predict(imagen)# image prediction
        y_predict = np.where(y_predict>0.5,1,0)

        return y_predict





In [ ]:
#